# Kaggle Colab AI Datasets training


### Kaggle:
Input data files are available in the _read-only_ "../input/" directory

You can write up to _20GB_ to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### Datasets:
[MultiSpeakerVoiceEN](https://www.kaggle.com/datasets/mfekadu/english-multispeaker-corpus-for-voice-cloning)
[CommonVoiceFR](https://www.kaggle.com/datasets/olmatz/commonvoicefr)

### Imports

In [2]:
import os
import time
import random
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

### English Multispeaker Dataset

In [24]:
!mkdir /kaggle/working/EN
!mkdir /kaggle/working/EN/train
!mkdir /kaggle/working/EN/test
!mkdir /kaggle/working/EN/model
!ls -a /kaggle/working/EN


In [25]:
DS_path = "../input/english-multispeaker-corpus-for-voice-cloning/VCTK-Corpus/VCTK-Corpus"
WAV_path = "../input/english-multispeaker-corpus-for-voice-cloning/VCTK-Corpus/VCTK-Corpus/wav48"
WAV_list = "../input/english-multispeaker-corpus-for-voice-cloning/VCTK-Corpus/VCTK-Corpus/speaker-info.txt"

__EN_working_path = "/kaggle/working/EN"
EN_train_path = os.path.join(__EN_working_path, "train")
EN_test_path = os.path.join(__EN_working_path, "test")
EN_model_path = os.path.join(__EN_working_path, "model")
print(EN_train_path, EN_test_path, EN_model_path)